In [1]:
#Importar librerias necesarias para el codigo
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
from datetime import datetime
import requests, os
import json
import shodan


In [2]:
 #configuracion de variables y conexion con cosmos
settings = {
       'host': os.environ.get('ACCOUNT_HOST', 'https://andresdiaz04.documents.azure.com:443/'),
    'master_key': os.environ.get('ACCOUNT_KEY', 'AplolgJ871casWDGMyaoCjSOkjOtRCEDC1kVhJdjE4Zr90XLSPUSJzC9dO3VvWN7djyrkprak7XGACDbc9Q4VQ=='),
    'database_id': os.environ.get('COSMOS_DATABASE', 'base2'),
    'container_id': os.environ.get('COSMOS_CONTAINER', 'prueba2')
}
#Asignación de valores a variables internas
HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']


#codigo para creal la base de datos y el contendero en azure cosmosdb
def delete_item(container, doc_id, account_number):
    print('\nDeleting Item by Id\n')

    response = container.delete_item(item=doc_id, partition_key=account_number)

    print('Deleted item\'s Id is {0}'.format(doc_id))

def read_item(container, doc_id, account_number):
    print('\nReading Item by Id\n')

    # We can do an efficient point read lookup on partition key and id
    response = container.read_item(item=doc_id, partition_key=account_number)

    print('Item read by Id {0}'.format(doc_id))
    print('Partition Key: {0}'.format(response.get('partitionKey')))
    print('Description: {0}'.format(response.get('VulnerabilityDesc')))
    
def read_items(container):
    print('\nReading all items in a container\n')

    item_list = list(container.read_all_items(max_item_count=10))

    print('Found {0} items'.format(item_list.__len__()))

    for doc in item_list:
        print('Item Id: {0}'.format(doc.get('id')))
        
def query_items(container, account_number):
    print('\nQuerying for an  Item by Partition Key\n')

    # Including the partition key value of account_number in the WHERE filter results in a more efficient query
    items = list(container.query_items(
        query="SELECT * FROM r WHERE r.partitionKey=@account_number",
        parameters=[
            { "name":"@account_number", "value": account_number }
        ]
    ))

    print('Item queried by Partition Key {0}'.format(items[0].get("id")))

def run_sample():
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")


if __name__ == '__main__':
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")


run_sample has caught an error. (Forbidden) Subscription owning the database account is disabled.
ActivityId: 60c38b47-c957-49a9-966c-71387a9f8abd, Microsoft.Azure.Documents.Common/2.14.0
Code: Forbidden
Message: Subscription owning the database account is disabled.
ActivityId: 60c38b47-c957-49a9-966c-71387a9f8abd, Microsoft.Azure.Documents.Common/2.14.0

run_sample done


In [3]:
import requests
import json

# Definir la URL de la API NVD para obtener información sobre una CVE específica
cve_url = "https://services.nvd.nist.gov/rest/json/cves/2.0?cveId=CVE-2019-1010218"

# Realizar una solicitud HTTP GET a la URL
response = requests.get(cve_url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Extraer contenido JSON de la respuesta
    cve_info = response.json()

    # Imprimir la información de la CVE formateada
    print("=== Información detallada de la CVE (CVE-2019-1010218) ===")
    print(json.dumps(cve_info, indent=4))
else:
    # En caso de error, mostrar un mensaje de error con el código de estado
    print("Error: No se pudo obtener la información de la CVE. Código de estado:", response.status_code)


Error: No se pudo obtener la información de la CVE. Código de estado: 503


In [4]:
import requests

# Definir la URL de la API NVD para obtener información sobre una CVE específica
url = "https://services.nvd.nist.gov/rest/json/cves/2.0?cveId=CVE-2019-1010218"

try:
    # Realizar una solicitud HTTP GET a la URL
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Extraer contenido JSON de la respuesta
        info_dict = response.json()

        # Obtener métricas CVSS versión 2.0 de la vulnerabilidad
        cvss_v2_metrics = info_dict['vulnerabilities'][0]['cve']['metrics']['cvssMetricV2']

        # Iterar sobre cada métrica CVSS versión 2.0 y mostrar el puntaje base y la severidad
        for metric in cvss_v2_metrics:
            base_score_v2 = metric['cvssData']['baseScore']
            severity_v2 = metric['baseSeverity']

            # Imprimir el puntaje base y la severidad de la métrica CVSS versión 2.0
            print(f"Puntaje base (CVSS 2.0): {base_score_v2} ({severity_v2})")
    else:
        # Si la solicitud no fue exitosa, mostrar el código de estado
        print(f"La solicitud no fue exitosa. Código de estado: {response.status_code}")
except Exception as e:
    # Si se produce un error durante la solicitud, mostrar un mensaje de error
    print("Error al obtener información de la CVE:", e)


La solicitud no fue exitosa. Código de estado: 503


In [ ]:
import json
import requests
from datetime import datetime

# IP de ejemplo
ip = "80.120.238.18"

# Función para consultar información detallada sobre una CVE en la base de datos del MITRE
def mitreCheckInformationbyCVE(CVE):
    url = f"https://cveawg.mitre.org/api/cve/{CVE}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error al obtener información de la CVE:", CVE)
        return None

# Función para obtener métricas CVSS relacionadas con una CVE desde la base de datos del NIST NVD
def get_cvss_metrics(cve_id):
    url = f"https://services.nvd.nist.gov/rest/json/cves/2.0?cveId={cve_id}"
    response = requests.get(url)
    if response.status_code == 200:
        info_dict = response.json()
        cvss_v2_metrics = info_dict['vulnerabilities'][0]['cve']['metrics']['cvssMetricV2']
        cvss_metrics_list = []
        for metric in cvss_v2_metrics:
            base_score_v2 = metric['cvssData']['baseScore']
            severity_v2 = metric['baseSeverity']
            cvss_metrics = {
                "Version": "2.0",
                "Base_Score": base_score_v2,
                "Severity": severity_v2
            }
            cvss_metrics_list.append(cvss_metrics)
        return cvss_metrics_list
    else:
        print("Error al obtener métricas CVSS para la CVE:", cve_id)
        return None

# Función para construir un objeto JSON con la información de una vulnerabilidad
def construct_json(item, cve_id, description, cvss_metrics):
    json_data = {
        'id': f"{item}_{datetime.now().strftime('%Y%m%d%H%M%S')}",
        'partitionKey': 'Account1',
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "CVE_ID": cve_id,
        "Description": description,
        "CVSS_Metrics": cvss_metrics
    }
    return json_data

# URL para obtener información de vulnerabilidades shodan en este caso específico
url = f"https://api.shodan.io/shodan/host/{ip}?key=1N4ar2xbSxr62Rw6cs3wI12Wqt2FijnW"

# Realizar la solicitud HTTP GET a la API de Shodan
response = requests.get(url)

# Convertir la respuesta JSON en un diccionario
response_dict = response.json()

# Obtener las vulnerabilidades en JSON
vulnerabilities = response_dict.get("vulns", [])

# Lista para almacenar las vulnerabilidades en formato JSON
vulnerability_data = []

# Iterar sobre cada CVE y obtener su descripción y métricas CVSS
for cve_id in vulnerabilities:
    print("Vulnerabilidad:", cve_id)
    
    # Obtener la descripción de la vulnerabilidad del API de MITRE
    cve_info = mitreCheckInformationbyCVE(cve_id)
    if cve_info:
        description = cve_info.get("description", "Descripción no disponible")
    else:
        description = "Descripción no disponible"
    print("Descripción:", description)
    
    # Obtener métricas CVSS de la vulnerabilidad
    cvss_metrics = get_cvss_metrics(cve_id)
    print("Métricas CVSS:", cvss_metrics)
    
    # Construir JSON para esta vulnerabilidad y agregarlo a la lista
    vulnerability_json = construct_json(f"InfoCVE-CVSS:{ip}", cve_id, description, cvss_metrics)
    vulnerability_data.append(vulnerability_json)
    print()  

# Imprimir el total de vulnerabilidades encontradas
total_vulnerabilities = len(vulnerabilities)
print(f"Total de vulnerabilidades encontradas para la IP {ip}: {total_vulnerabilities}")

# Convertir la lista de vulnerabilidades a JSON
final_json = json.dumps(vulnerability_data, indent=4)
print("JSON final:")
print(final_json)




Vulnerabilidad: CVE-2018-10549
Descripción: Descripción no disponible
